<a href="https://colab.research.google.com/github/locastre/octave-colab/blob/master/updated_DL_CERR_routines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Deep learning-based segmentation of ~~Heart Sub-Structure~~ Chewing and Swallowing Structures

This notebook demonstrates the application of a deep neural network model on CERR planning CT

---



### Install Octave 6.2

In [ ]:
#install library dependencies
%%capture
! apt-get update
! apt-get install libgraphicsmagick++1-dev libsuitesparse-dev libqrupdate1 libfftw3-3 gnuplot
! pip install octave_kernel
! pip install oct2py

In [ ]:
# download compiled octave package (latest)
%%capture
import urllib.request, json, os
os.chdir('/content')
with urllib.request.urlopen("https://api.github.com/repos/cerr/octave-colab/releases/latest") as url:
    data = json.loads(url.read().decode())
fname = data['assets'][0]['name']
requrl = data['assets'][0]['browser_download_url']
urllib.request.urlretrieve(requrl, fname)

In [ ]:
# unzip, identify octave folder name
!tar xf {fname}
top_folder = !tar tf {fname} | head -1
octave_path = top_folder[0][:-1]
print(octave_path)

octave6.2.0


In [ ]:
import os
os.environ['OCTAVE_EXECUTABLE'] = '/content/' + octave_path + '/bin/octave-cli'
os.environ['PATH'] = '/content/' + octave_path + '/bin:' + os.environ['PATH']

In [ ]:
import oct2py
import octave_kernel
from oct2py import octave
%load_ext oct2py.ipython

In [ ]:
%%octave

warning('off','all')

In [ ]:
%%octave

pkg load image
pkg load io
pkg load statistics

In [ ]:
#clean up
os.chdir('/content')
os.remove(fname)

### Download the pretrained models

In [ ]:
os.chdir('/content')
!git clone https://github.com/cerr/CT_SwallowingAndChewing_DeepLabV3.git
with urllib.request.urlopen("https://api.github.com/repos/cerr/CT_SwallowingAndChewing_DeepLabV3/releases/latest") as url:
    data = json.loads(url.read().decode())
tag_name = data['tag_name']
!cd CT_SwallowingAndChewing_DeepLabV3 && git checkout {tag_name}

fatal: destination path 'CT_SwallowingAndChewing_DeepLabV3' already exists and is not an empty directory.
HEAD is now at f325500c Merge pull request #8 from aditiiyer/Windows


### Download CERR

In [ ]:
os.chdir('/content')
#commit_hash = '26587ebd684629ecda517e2d6bb294b10fffb35e'
commit_hash = '3a13b87'
!git clone https://github.com/cerr/CERR.git
!cd /content/CERR && git checkout octave_dev && git checkout {commit_hash}

fatal: destination path 'CERR' already exists and is not an empty directory.
Checking out files: 100% (475/475), done.
Previous HEAD position was 3a13b876 Added resampled H&N dataset
Switched to branch 'octave_dev'
Your branch is up to date with 'origin/octave_dev'.
Note: checking out '3a13b87'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 3a13b876 Added resampled H&N dataset


### Create conda environment for model and its dependencies

In [ ]:
%%capture
os.chdir('/content')
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
%%capture
!conda env create -f /content/CT_SwallowingAndChewing_DeepLabV3/environment.yml

### Apply segmentation models to sample data

In [ ]:
octave.addpath(octave.genpath('/content/CERR'))

'.:/content/CERR:/content/octave6.2.0/share/octave/packages/statistics-1.4.2/base:/content/octave6.2.0/share/octave/packages/statistics-1.4.2/distributions:/content/octave6.2.0/share/octave/packages/statistics-1.4.2/models:/content/octave6.2.0/share/octave/packages/statistics-1.4.2/tests:/content/octave6.2.0/share/octave/packages/statistics-1.4.2:/content/octave6.2.0/lib/octave/packages/io-2.6.3/x86_64-pc-linux-gnu-api-v55:/content/octave6.2.0/share/octave/packages/io-2.6.3:/content/octave6.2.0/lib/octave/packages/image-2.12.0/x86_64-pc-linux-gnu-api-v55:/content/octave6.2.0/share/octave/packages/image-2.12.0:/usr/local/lib/python3.7/dist-packages/oct2py:/usr/local/lib/python3.7/dist-packages/octave_kernel:/content/octave6.2.0/lib/octave/6.2.0/site/oct/x86_64-pc-linux-gnu:/content/octave6.2.0/lib/octave/site/oct/api-v55/x86_64-pc-linux-gnu:/content/octave6.2.0/lib/octave/site/oct/x86_64-pc-linux-gnu:/content/octave6.2.0/share/octave/6.2.0/site/m:/content/octave6.2.0/share/octave/site/a

In [ ]:
sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/' + \
              'CERR_plans/head_neck_ex1_20may03_resamp.mat.bz2'
%octave_push sampleData
planC = octave.loadPlanC(sampleData, octave.tempdir())
planC = octave.updatePlanFields(planC)
planC = octave.quality_assure_planC(sampleData, planC)

algorithm = 'CT_ChewingStructures_DeepLabV3^CT_Larynx_DeepLabV3^' + \
            'CT_PharyngealConstrictor_DeepLabV3';
sessionPath = '/content/temp';
functionName = '/content/CT_SwallowingAndChewing_DeepLabV3/' + \
               'model_wrapper/CT_ChewingStructures_DeepLabV3/chewing_main.py^' + \
               '/content/CT_SwallowingAndChewing_DeepLabV3/' + \
               'model_wrapper/CT_Larynx_DeepLabV3/larynx_main.py^' + \
               '/content/CT_SwallowingAndChewing_DeepLabV3/' + \
               'model_wrapper/CT_PharyngealConstrictor_DeepLabV3/' + \
               'pharyngeal_constrictor_main.py';
condaEnvName = 'swChewEnv'

CERR>>  Decompressing head_neck_ex1_20may03_resamp.mat.bz2...

ans = 0
CERR>>  Loading head_neck_ex1_20may03_resamp.mat.bz2...
CERR>>  Loaded head_neck_ex1_20may03_resamp.mat.bz2...


In [ ]:
%%capture

planC = octave.runSegForPlanCInCondaEnv(planC,sessionPath,algorithm, \
                                        functionName,[],condaEnvName);

### Display segmentations

In [ ]:
from oct2py import octave
%octave_push planC

In [ ]:
%%octave

addpath(genpath('/content/CERR'))

%Get scan array
indexS = planC{end};
scanNum = 1;
ctOffset = planC{indexS.scan}(scanNum).scanInfo(1).CTOffset;
scanArray = single(getScanArray(scanNum,planC)) - ctOffset;

%Get structure labels & masks
numStructs = length(planC{indexS.structures});
structNameC = {planC{indexS.structures}.structureName};
strNameC = {'Left_masseter', 'Right_masseter', 'Left_medial_pterygoid',...
              'Right_medial_pterygoid', 'Larynx_DLabV3', 'Constrictor_muscle'};
for strNum = 1:length(strNameC)
    strx = strNameC{strNum};
    idx = getMatchingIndex(strx,structNameC,'EXACT');
    mask3M = getStrMask(idx, planC);
    maskC{strNum} = mask3M;
end

In [ ]:
%octave_pull maskC strNameC scanArray

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets

dx, dy = 1, 1

x = np.arange(0, 255, dx)
y = np.arange(0, 255, dy)
extent = np.min(x), np.max(x), np.min(y), np.max(y)

clear_output(wait=True)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def show_axial_slice(slcNum):
    clear_output(wait=True)
    print('Slice '+str(slcNum))
    if 'fig' in locals():
        fig.remove()
    fig, ax = plt.subplots()
    window_center = 45
    window_width = 125
    windowed_img = window_image(scanArray[:,:,slcNum-1],
                                window_center,window_width)
    im1 = ax.imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='nearest', extent=extent)
    
    cmaps = [plt.cm.Oranges,plt.cm.Oranges,plt.cm.Blues,plt.cm.Blues, \
             plt.cm.Purples,plt.cm.Greens]

    for maskNum in range(0,6,1):
        mask_cmap = cmaps[maskNum]
        mask_cmap.set_under('k', alpha=0)
        im2 = ax.imshow(maskC[0,maskNum][:,:,slcNum-1], 
                        cmap=mask_cmap, alpha=.8, extent=extent,
                        interpolation='none', clim=[0.5, 1])        
    plt.show()

slice_slider = widgets.IntSlider(min=1,max=79,step=1)
outputSlc = widgets.Output()

display(slice_slider, outputSlc)

def update_slice(change):
    with outputSlc:
        show_axial_slice(change['new'])

slice_slider.observe(update_slice, names='value')
